<a href="https://colab.research.google.com/github/wairiukoirwine/ML11/blob/main/full_proof_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ecitizen_ml_pipeline.py
import io
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, f1_score
import matplotlib.pyplot as plt

# ===================== HELPER FUNCTIONS =====================
def load_csv(file_path):
    """Load CSV into DataFrame"""
    df = pd.read_csv(file_path)
    return df

def infer_problem_type(y_series):
    """Detect if the target is regression or classification"""
    if pd.api.types.is_numeric_dtype(y_series):
        return 'regression' if y_series.nunique() > 5 else 'classification'
    else:
        return 'classification'

def build_preprocessor(X):
    numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
    categorical_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()

    numeric_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

    preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

    return preprocessor

def train_and_evaluate(df, feature_cols, target_col, model_choice='linear'):
    X = df[feature_cols]
    y = df[target_col]
    problem_type = infer_problem_type(y)
    print(f"Detected problem type: {problem_type}")

    preprocessor = build_preprocessor(X)

    if model_choice.lower() == 'linear':
        model = LinearRegression()
    else:
        model = DecisionTreeRegressor() if problem_type == 'regression' else DecisionTreeClassifier()

    pipeline = Pipeline([('preprocessor', preprocessor), ('model', model)])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    print("\n=== MODEL EVALUATION ===")
    if problem_type == 'regression':
        print(f"MSE: {mean_squared_error(y_test, y_pred)}")
        print(f"R2: {r2_score(y_test, y_pred)}")
    else:
        print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
        print(f"F1 Score: {f1_score(y_test, y_pred, average='weighted')}")

    # Save predictions
    pred_df = X_test.copy()
    pred_df['Actual'] = y_test.values
    pred_df['Predicted'] = y_pred
    pred_df.to_csv("predictions.csv", index=False)
    print("Predictions saved as predictions.csv")

    # Plot first feature vs actual/predicted
    plt.figure(figsize=(10,5))
    first_feature = feature_cols[0]
    unique_values = np.sort(X_test[first_feature].unique())
    actual_means = [y_test[X_test[first_feature] == val].mean() for val in unique_values]
    predicted_means = [y_pred[X_test[first_feature] == val].mean() for val in unique_values]

    width = 0.4
    plt.bar(np.arange(len(unique_values)) - width/2, actual_means, width=width, label='Actual', color='blue')
    plt.bar(np.arange(len(unique_values)) + width/2, predicted_means, width=width, label='Predicted', color='red')
    plt.xticks(np.arange(len(unique_values)), unique_values)
    plt.xlabel(first_feature)
    plt.ylabel(target_col)
    plt.title('Actual vs Predicted by Feature')
    plt.legend()
    plt.show()

# ===================== MAIN BLOCK FOR PREVIEW =====================
if __name__ == "__main__":
    print("This is a preview version of the ML pipeline.")
    print("To run it interactively, load a CSV and call `train_and_evaluate(df, features, target, model)`")


FileUpload(value={}, accept='.csv', description='Upload')

SelectMultiple(description='Features', options=(), value=())

Dropdown(description='Target', options=(), value=None)

Dropdown(description='Model', options=('linear', 'tree'), value='linear')

Button(description='Run ML Pipeline', style=ButtonStyle())

Output()